# 동국대학교 공지사항 크롤링 노트북
동국대학교 대표 홈페이지에서 학사/장학 공지를 페이지 단위로 모두 수집한 뒤,
공지 본문과 첨부파일 정보까지 함께 데이터프레임으로 정리하는 예제입니다.
필요한 패키지를 설치한 뒤 아래 셀을 순서대로 실행하세요.

> ⚠️ 이미지 공지 OCR을 사용하려면 로컬에 Tesseract OCR 엔진과 한글 데이터(`kor`)가 설치되어 있어야 합니다. 설치되어 있지 않으면 이미지 텍스트는 건너뜁니다.


In [18]:
import re
import time
import base64
from datetime import datetime
from typing import Any, Dict, List, Optional
from urllib.parse import quote, urljoin
from pathlib import Path

import pandas as pd
import requests
from bs4 import BeautifulSoup, FeatureNotFound
from bs4.builder import ParserRejectedMarkup
from io import BytesIO

import pytesseract

from PIL import Image


In [19]:
BASE_URL = "https://www.dongguk.edu"
BOARD_CODES = {
    "일반공지": "GENERALNOTICES",
    "학사공지": "HAKSANOTICE",
    "장학공지": "JANGHAKNOTICE",
    "입학공지": "IPSINOTICE",
    "국제공지": "GLOBALNOLTICE",
    "학술공지": "HAKSULNOTICE",
    "안전공지": "SAFENOTICE",
    "행사공지": "BUDDHISTEVENT",
}
HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; DonggukNoticeCrawler/0.2)",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
}
SELECT_COLUMNS = [
    "board_name",
    "title",
    "category",
    "posted_at",
    "is_pinned",
    "detail_url",
    "content_text",
    "attachments",
]
COLUMN_LABELS = {
    "board_name": "게시판",
    "title": "제목",
    "category": "카테고리",
    "posted_at": "게시일",
    "is_pinned": "상단고정",
    "detail_url": "상세URL",
    "content_text": "본문",
    "attachments": "첨부파일",
}


In [20]:
TARGET_BOARDS = ["일반공지", "학사공지", "장학공지", "입학공지", "국제공지", "학술공지", "안전공지", "행사공지"]  # 수집 대상 게시판
MAX_PAGES = 10  # 게시판별 최대 조회 페이지 수
REQUEST_DELAY = 0.2  # 상세 페이지 요청 간 대기 시간(초)
OUTPUT_DIR = Path("dongguk_notice_csv")



In [ ]:
PARSER_CANDIDATES = ("lxml", "html5lib", "html.parser")
HWPJSON_SECTION_PATTERN = re.compile(r"<!\[[^<]*?data-hwpjson.*?\]>", re.IGNORECASE | re.DOTALL)
FILENAME_SANITIZE_PATTERN = re.compile(r"[^0-9A-Za-z가-힣_-]+")

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def strip_hwpjson_sections(markup: str) -> str:
    cleaned = markup
    while True:
        updated = HWPJSON_SECTION_PATTERN.sub("", cleaned)
        if updated == cleaned:
            break
        cleaned = updated
    if 'data-hwpjson' in cleaned.lower():
        cleaned = re.sub(r'<!\[\s*data-hwpjson', '<![CDATA', cleaned, flags=re.IGNORECASE)
    return cleaned

def neutralize_marked_sections(markup: str) -> str:
    def replacer(match: re.Match) -> str:
        segment = match.group(0)
        return f"<!--{segment[2:-1]}-->"
    return re.sub(r'<!\[[^>]*?\]>', replacer, markup, flags=re.DOTALL)

def make_soup(markup: str) -> BeautifulSoup:
    cleaned_markup = strip_hwpjson_sections(markup)
    last_exc: Optional[Exception] = None

    for parser in PARSER_CANDIDATES:
        try:
            return BeautifulSoup(cleaned_markup, parser)
        except (FeatureNotFound, ParserRejectedMarkup) as exc:
            last_exc = exc
            continue
        except Exception as exc:
            last_exc = exc
            continue

    fallback_markup = neutralize_marked_sections(cleaned_markup)
    for parser in PARSER_CANDIDATES:
        try:
            return BeautifulSoup(fallback_markup, parser)
        except (FeatureNotFound, ParserRejectedMarkup):
            continue
        except Exception:
            continue

    if last_exc is not None:
        raise ParserRejectedMarkup(f"HTML 파싱 실패: {last_exc}") from last_exc
    raise RuntimeError("No HTML parser could parse the provided markup.")

def sanitize_filename_component(value: Optional[str], fallback: str = "notice") -> str:
    if not value:
        return fallback
    cleaned = FILENAME_SANITIZE_PATTERN.sub("_", value).strip("_")
    return (cleaned or fallback)[:80]

def save_notice_csv(record: Dict[str, Any]) -> Path:
    board_dir = OUTPUT_DIR / sanitize_filename_component(record.get("board_name"), "board")
    board_dir.mkdir(parents=True, exist_ok=True)

    detail_url = record.get("detail_url") or ""
    article_id = str(record.get("article_id") or "").strip()
    if not article_id and isinstance(detail_url, str):
        article_id = detail_url.rstrip("/").split("/")[-1]

    identifier = sanitize_filename_component(article_id, datetime.now().strftime("%Y%m%d%H%M%S%f"))
    title_component = sanitize_filename_component(record.get("title"), "notice")

    file_path = board_dir / f"{identifier}_{title_component}.csv"

    notice_row = {key: record.get(key) for key in SELECT_COLUMNS}
    notice_df = pd.DataFrame([notice_row])
    notice_df.rename(columns=COLUMN_LABELS, inplace=True)
    # notice_df.to_csv(file_path, index=False, encoding='utf-8-sig')

    return file_path


In [22]:
def fetch_notice_list(board_code: str, page: int = 1) -> List[Dict[str, Any]]:
    """지정된 공지 게시판의 목록 페이지에서 기사 ID 등 메타데이터만 추출합니다."""
    url = f"{BASE_URL}/article/{board_code}/list"
    response = requests.get(url, params={"pageIndex": page}, headers=HEADERS, timeout=10)
    response.raise_for_status()

    soup = make_soup(response.text)
    notices: List[Dict[str, Any]] = []

    for item in soup.select("div.board_list > ul > li"):
        anchor = item.find("a")
        if anchor is None:
            continue

        onclick = anchor.get("onclick", "")
        match = re.search(r"goDetail\((\d+)\)", onclick)
        if match is None:
            continue
        article_id = int(match.group(1))

        title_tag = anchor.select_one("p.tit")
        title = title_tag.get_text(" ", strip=True) if title_tag else ""

        category_tag = anchor.select_one("div.top > em")
        category = category_tag.get_text(strip=True) if category_tag else None

        info_spans = anchor.select("div.info span")
        posted_at: Optional[datetime.date] = None
        views: Optional[int] = None
        if info_spans:
            raw_date = info_spans[0].get_text(strip=True).rstrip(".")
            try:
                posted_at = datetime.strptime(raw_date, "%Y.%m.%d").date()
            except ValueError:
                posted_at = None
        if len(info_spans) > 1:
            match_views = re.search(r"(\d+)", info_spans[1].get_text(strip=True))
            if match_views:
                views = int(match_views.group(1))

        is_pinned = anchor.select_one("div.mark span.fix") is not None

        notices.append({
            "article_id": article_id,
            "title": title,
            "category": category,
            "posted_at": posted_at,
            "views": views,
            "is_pinned": is_pinned,
        })

    return notices


In [23]:
def fetch_notice_detail(board_code: str, article_id: int) -> Dict[str, Any]:
    """특정 공지 글의 본문과 첨부파일 목록을 추출합니다."""
    url = f"{BASE_URL}/article/{board_code}/detail/{article_id}"
    response = requests.get(url, headers=HEADERS, timeout=10)
    response.raise_for_status()

    soup = make_soup(response.text)
    container = soup.select_one("div.board_view")
    if container is None:
        raise RuntimeError("상세 정보를 찾을 수 없습니다.")

    title_tag = container.select_one("div.tit > p")
    title_text = title_tag.get_text(strip=True) if title_tag else ""

    info_block = container.select_one("div.tit > div.info")
    posted_at = None
    views = None
    if info_block:
        for span in info_block.select("span"):
            text = span.get_text(strip=True)
            if text.startswith("등록일"):
                raw_date = text.replace("등록일", "").strip().rstrip(".")
                try:
                    posted_at = datetime.strptime(raw_date, "%Y.%m.%d").date()
                except ValueError:
                    posted_at = None
            elif text.startswith("조회"):
                match_views = re.search(r"(\d+)", text)
                if match_views:
                    views = int(match_views.group(1))

    content_block = container.select_one("div.view_cont")
    if content_block:
        for script in content_block.find_all("script"):
            script.decompose()
        content_html = content_block.decode_contents().strip()
        content_text = content_block.get_text("\n", strip=True)
        inline_images = [img.get("src") for img in content_block.find_all("img")]
    else:
        content_html = ""
        content_text = ""
        inline_images = []

    attachments: List[Dict[str, Any]] = []
    for link in container.select("div.view_files ul li a"):
        href = link.get("href", "")
        match = re.search(r"downGO\('(.+?)','(.+?)','(.+?)'\)", href)
        if not match:
            continue
        name, path, stored = match.groups()
        download_url = urljoin(
            BASE_URL,
            f"/cmmn/fileDown.do?filename={quote(name)}&filepath={quote(path, safe='/')}&filerealname={quote(stored)}"
        )
        attachments.append({
            "name": name,
            "url": download_url,
        })

    image_attachment_sources: List[str] = []
    for attachment in attachments:
        name_lower = attachment["name"].lower()
        if name_lower.endswith((".png", ".jpg", ".jpeg", ".gif", ".bmp", ".webp")):
            image_attachment_sources.append(attachment["url"])

    return {
        "title": title_text,
        "posted_at": posted_at,
        "views": views,
        "content_html": content_html,
        "content_text": content_text,
        "attachments": attachments,
        "detail_url": url,
    }



In [24]:
def collect_board(board_name: str, board_code: str, max_pages: Optional[int] = None, delay: float = 0.2) -> pd.DataFrame:
    """지정된 게시판의 공지를 순회하며 선택한 열만 담은 DataFrame을 반환합니다."""
    records: List[Dict[str, Any]] = []
    seen_ids = set()
    page = 1

    while True:
        if max_pages is not None and page > max_pages:
            break

        notice_list = fetch_notice_list(board_code, page=page)
        if not notice_list:
            break

        for meta in notice_list:
            article_id = meta["article_id"]
            if article_id in seen_ids:
                continue
            seen_ids.add(article_id)

            detail = fetch_notice_detail(board_code, article_id)

            record = {
                "board_name": board_name,
                "board_code": board_code,
                "article_id": article_id,
                "title": meta.get("title"),
                "category": meta.get("category"),
                "posted_at": detail.get("posted_at") or meta.get("posted_at"),
                "views": detail.get("views") or meta.get("views"),
                "is_pinned": meta.get("is_pinned"),
                "detail_url": detail.get("detail_url"),
                "content_html": detail.get("content_html"),
                "content_text": detail.get("content_text"),
                "attachments": detail.get("attachments"),
            }
            records.append(record)
            try:
                save_notice_csv(record)
            except Exception as exc:
                print(f'⚠️ CSV 저장 실패: {board_name} {article_id}: {exc}')

            if delay:
                time.sleep(delay)

        print(f"{board_name} {page}페이지 완료 (누적 {len(records)}건)")
        page += 1

    if not records:
        empty_columns = [COLUMN_LABELS[col] for col in SELECT_COLUMNS]
        return pd.DataFrame(columns=empty_columns)

    df = pd.DataFrame(records)
    df["posted_at"] = pd.to_datetime(df["posted_at"], errors="coerce").dt.date
    df.sort_values(by=["posted_at", "article_id"], ascending=[False, False], inplace=True)
    df.reset_index(drop=True, inplace=True)

    selected = df[SELECT_COLUMNS].copy()
    selected.rename(columns=COLUMN_LABELS, inplace=True)
    return selected


In [25]:
dataframes: List[pd.DataFrame] = []
for board_name in TARGET_BOARDS:
    board_code = BOARD_CODES.get(board_name)
    if not board_code:
        print(f"⚠️ 게시판 코드를 찾을 수 없습니다: {board_name}")
        continue
    df = collect_board(board_name, board_code, max_pages=MAX_PAGES, delay=REQUEST_DELAY)
    dataframes.append(df)

if dataframes:
    all_notices_df = pd.concat(dataframes, ignore_index=True)
else:
    all_notices_df = pd.DataFrame(columns=[COLUMN_LABELS[col] for col in SELECT_COLUMNS])

print(f"총 {len(all_notices_df)}건의 공지 수집")



일반공지 1페이지 완료 (누적 10건)
일반공지 2페이지 완료 (누적 20건)
일반공지 3페이지 완료 (누적 30건)
일반공지 4페이지 완료 (누적 40건)
일반공지 5페이지 완료 (누적 50건)
일반공지 6페이지 완료 (누적 60건)
일반공지 7페이지 완료 (누적 70건)
일반공지 8페이지 완료 (누적 80건)
일반공지 9페이지 완료 (누적 90건)
일반공지 10페이지 완료 (누적 100건)
학사공지 1페이지 완료 (누적 11건)
학사공지 2페이지 완료 (누적 20건)
학사공지 3페이지 완료 (누적 30건)
학사공지 4페이지 완료 (누적 40건)
학사공지 5페이지 완료 (누적 50건)
학사공지 6페이지 완료 (누적 60건)
학사공지 7페이지 완료 (누적 70건)
학사공지 8페이지 완료 (누적 80건)
학사공지 9페이지 완료 (누적 90건)
학사공지 10페이지 완료 (누적 100건)
장학공지 1페이지 완료 (누적 17건)
장학공지 2페이지 완료 (누적 26건)
장학공지 3페이지 완료 (누적 36건)
장학공지 4페이지 완료 (누적 44건)
장학공지 5페이지 완료 (누적 53건)
장학공지 6페이지 완료 (누적 60건)
장학공지 7페이지 완료 (누적 70건)
장학공지 8페이지 완료 (누적 80건)
장학공지 9페이지 완료 (누적 90건)
장학공지 10페이지 완료 (누적 100건)
입학공지 1페이지 완료 (누적 10건)
입학공지 2페이지 완료 (누적 20건)
입학공지 3페이지 완료 (누적 30건)
입학공지 4페이지 완료 (누적 40건)
입학공지 5페이지 완료 (누적 50건)
입학공지 6페이지 완료 (누적 60건)
입학공지 7페이지 완료 (누적 70건)
입학공지 8페이지 완료 (누적 80건)
입학공지 9페이지 완료 (누적 90건)
입학공지 10페이지 완료 (누적 100건)
국제공지 1페이지 완료 (누적 13건)
국제공지 2페이지 완료 (누적 21건)
국제공지 3페이지 완료 (누적 30건)
국제공지 4페이지 완료 (누적 40건)
국제공지 5페이지 완료 (누적 50건)
국제

In [29]:
all_notices_df.head()


,게시판,제목,카테고리,게시일,상단고정,상세URL,본문,첨부파일
0,일반공지,공지 [교수학습혁신센터] 2025-2학기 협력학습 동국 튜터링 최종 선정 튜터 발표,None,2025-09-29,True,https://www.dongguk.edu/article/GENERALNOTICES...,,"[{'name': '(붙임1) 2025-2학기 최종 선정 튜터 명단.pdf', 'u..."
1,일반공지,동국대 캠퍼스타운 창업특강 4·5차 안내(4차 : 선배창업가 고요한택시 송민표 대표...,None,2025-09-29,False,https://www.dongguk.edu/article/GENERALNOTICES...,🚀\n[\n동국대 캠퍼스타운 창업특강\n4·5\n차 안내\n]\n동국대학교 출신 선...,[]
2,일반공지,공지 2025년 추석연휴 임시휴업일 지정 안내,None,2025-09-29,True,https://www.dongguk.edu/article/GENERALNOTICES...,2025년 추석연휴 임시휴업일 지정 안내\n2025년 추석 연휴 관련 임시휴업일 지...,[]
3,일반공지,공지 [학생역량개발팀] 제 20기 동국108리더스 모집 안내,None,2025-09-29,True,https://www.dongguk.edu/article/GENERALNOTICES...,제 20기 동국108리더스를 모집합니다.\n많은 관심과 지원 바랍니다.\n지원링크 ...,"[{'name': '제20기 동국108리더스 이력서 및 자기소개서 양식.hwp', ..."
4,일반공지,공지 [카운슬링센터] 온라인 심리검사 프로그램 참여 안내: '나의 대학생활과 학습습...,None,2025-09-26,True,https://www.dongguk.edu/article/GENERALNOTICES...,,[]


In [30]:
# 공지별 CSV 저장 상태를 요약합니다.
total_notices = len(all_notices_df)
print(f"총 {total_notices}건의 공지를 {OUTPUT_DIR} 경로에 공지별 CSV로 저장했습니다.")
all_notices_df.head(3)


총 739건의 공지를 dongguk_notice_csv 경로에 공지별 CSV로 저장했습니다.


,게시판,제목,카테고리,게시일,상단고정,상세URL,본문,첨부파일
0,일반공지,공지 [교수학습혁신센터] 2025-2학기 협력학습 동국 튜터링 최종 선정 튜터 발표,None,2025-09-29,True,https://www.dongguk.edu/article/GENERALNOTICES...,,"[{'name': '(붙임1) 2025-2학기 최종 선정 튜터 명단.pdf', 'u..."
1,일반공지,동국대 캠퍼스타운 창업특강 4·5차 안내(4차 : 선배창업가 고요한택시 송민표 대표...,None,2025-09-29,False,https://www.dongguk.edu/article/GENERALNOTICES...,🚀\n[\n동국대 캠퍼스타운 창업특강\n4·5\n차 안내\n]\n동국대학교 출신 선...,[]
2,일반공지,공지 2025년 추석연휴 임시휴업일 지정 안내,None,2025-09-29,True,https://www.dongguk.edu/article/GENERALNOTICES...,2025년 추석연휴 임시휴업일 지정 안내\n2025년 추석 연휴 관련 임시휴업일 지...,[]


In [ ]:
all_notices_df.to_csv('./data/dongguk_notices.csv', index=False, encoding='utf-8-sig')